# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# Import the Configure gmaps, apply Google Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file from the output_data folder under WeatherPy and display
csvpath = '../output_data/cities.csv'
cities_df = pd.read_csv(csvpath, index_col=0)
cities_df

,City,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,albany,42.6001,-73.9662,77.58,57,55,6.82,US,1627078986
1,bredasdorp,-34.5322,20.0403,49.32,76,0,10.36,ZA,1627078907
2,port alfred,-33.5906,26.8910,53.26,66,100,4.43,ZA,1627078919
3,ushuaia,-54.8000,-68.3000,37.06,81,40,9.22,AR,1627078762
4,agua de dios,4.3765,-74.6700,87.82,59,99,5.37,CO,1627078993
...,...,...,...,...,...,...,...,...,...
566,katobu,-4.9408,122.5276,73.63,88,84,3.24,ID,1627079750
567,madaoua,14.0730,5.9600,81.59,66,62,13.98,NE,1627079751
568,trat,12.5000,102.5000,76.01,95,100,6.38,TH,1627079752
569,udachnyy,66.4167,112.4000,62.01,77,39,15.43,RU,1627079610


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# NOTE:  Configure gmaps was already added in the first cell

In [4]:
# Use Latitude and Longitude as the locations, and humidity as weights for the heat layer
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity (%)'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Latitude,571.0,2.091748e+01,32.628689,-5.480000e+01,-5.592500e+00,2.228550e+01,4.900810e+01,7.821860e+01
Longitude,571.0,1.530426e+01,90.268919,-1.791667e+02,-6.775360e+01,1.842320e+01,9.547260e+01,1.793167e+02
Max Temperature,571.0,7.018536e+01,14.375559,3.308000e+01,5.942000e+01,7.300000e+01,8.072000e+01,1.062500e+02
Humidity (%),571.0,7.149737e+01,19.184092,9.000000e+00,6.300000e+01,7.600000e+01,8.500000e+01,1.000000e+02
Cloudiness (%),571.0,5.291419e+01,37.981593,0.000000e+00,1.400000e+01,5.900000e+01,9.000000e+01,1.000000e+02
Wind Speed (MPH),571.0,8.269632e+00,6.013416,0.000000e+00,3.850000e+00,6.910000e+00,1.137500e+01,3.528000e+01
Date,571.0,1.627079e+09,237.447542,1.627079e+09,1.627079e+09,1.627079e+09,1.627080e+09,1.627080e+09


In [6]:
# DataFrame with filtered weather conditions
# Narrow down the DataFrame to find ideal weather conditions
# Max temp lower than 80 but higher than 70
# Zero cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_cities_df = cities_df.loc[(cities_df['Max Temperature'] > 70)  &
                                (cities_df['Max Temperature'] < 80) & 
                                (cities_df['Wind Speed (MPH)'] < 50) &
                                (cities_df['Cloudiness (%)'] == 32)
                       ]

# If is null value, drop it
hotel_df = ideal_cities_df.dropna()

# Print out the number of cities and display the dataframe
print(f'There are {len(hotel_df)} ideal cities for vacation')
hotel_df

There are 1 ideal cities for vacation


,City,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
58,yining,29.0361,114.5636,74.07,98,32,3.04,CN,1627079067


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create search list with converted latitude and Longitude
Latitude = hotel_df['Latitude'].tolist()
Longitude = hotel_df['Longitude'].tolist()
hotel_lst = []

# Gather first hotel tfor each cities
for i in range(len(hotel_df)):
    target_coordinates = f"{Latitude[i]}, {Longitude[i]}"
    target_search = 'hotel'
    target_type = "hotel"
    target_radius = 5000

      # Set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "types": target_type,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Apply the try and except block, code won't stop when no hotel isn't available
    
    try:
        
        # Run a request using params dictionary
        response = requests.get(base_url, params=params).json()
        hotel_lst.append(response['results'][0]['name'])
    
    except:
        
        # Append the string to indicate no hotel was found
        hotel_lst.append('NO HOTEL FOUND')
        
# After the loop, add the hotel list as a column in the dataframe
hotel_df['Hotel Name'] = hotel_lst

In [8]:
# Display the dataframe
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
58,yining,29.0361,114.5636,74.07,98,32,3.04,CN,1627079067,NO HOTEL FOUND


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))